# Bigram Language Models

In this notebook, we are going to implement a bigram language model and use interpolation for smoothing.

The bigram probabilities will be estimated by using MLE as follows:

$$
P_{ML}(w_i|w_{i-1})=\frac{c(w_{i-1}w_{i})}{c(w_{i-1})}
$$

Recall that, the smoothed bigram probabilities using interpolation technique is calculated as follows.

Bigrams:

$$
P(w_i|w_{i-1})=\lambda_2 \times P_{ML}(w_i|w_{i-1})+(1-\lambda_2)\times P(w_i)
$$

where $P(w_i)$ is the smoothed unigram probability and calculated as follows.

$$
P(w_i)=\lambda_1 \times P_{ML}(w_i) + (1-\lambda_1) \times \frac{1}{N}
$$

where $N$ is a large number, e.g., $N=1,000,000$.

There are two parts in this programming assignment.

- Training: you will estimate unigram and bigram probabilities from a raw text data file and save parameters of the language model into a file.
- Test: you use the trained language model to calculate calculates entropy, perplexity on the test data. You will need to use interpolation to calculate smoothed probabilities in testing.

Please refer the pseudo-code in slide 17, 18 of the lecture [NLP Programming Tutorial 2 -Bigram Language Models](http://www.phontron.com/slides/nlp-programming-en-02-bigramlm.pdf) to complete the assignment.

## Data

We will use the file [wiki-en-train.word](https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word) as the training data, and [wiki-en-test.
word](https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word) as the test data. To test our implementation quickly, we will use small data file [02-train-input.txt](https://github.com/neubig/nlptutorial/blob/master/test/02-train-input.txt). All data files are from the [nlptutorial](https://github.com/neubig/nlptutorial) by Graham Neubig.

As the first step, we will download all necessary data files using `wget` command line.

In [19]:
# !rm -f wiki-en-train.word
# !wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word

# !rm -f wiki-en-test.word
# !wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word

# !rm -f 02-train-input.txt
# !wget https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-input.txt

In [20]:
import urllib.request
import os

files = {
    "wiki-en-train.word": "https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word",
    "wiki-en-test.word": "https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word",
    "02-train-input.txt": "https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-input.txt",
}

for filename, url in files.items():
    if os.path.exists(filename):
        os.remove(filename)
    urllib.request.urlretrieve(url, filename)
    print(f"Downloaded {filename}")


Downloaded wiki-en-train.word
Downloaded wiki-en-test.word
Downloaded 02-train-input.txt


## Part 1: Estimating probabilities

What you need to do in this part is complete the function `train_bigram` to estimate unigram, bigram probabilities (using MLE method) from a text file and save probabilities to a model file.

The format of the model file is as follows. Each line includes an n-gram (unigram or bigram) and its probability estimated by MLE method.

```
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
...
```

### Part 1.1: Function train_bigram()

In [33]:
from collections import defaultdict


def train_bigram(train_file, model_file):
    """Train bigram language model and save to model file
    """
    counts = defaultdict(int)  # count the n-gram
    context_counts = defaultdict(int)   # count the context
    with open(train_file, encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')

            for i in range(1, len(words)):  # Note: starting at 1, after <s>
                # TODO: Write code to count bigrams and their contexts
                # YOUR CODE HERE
                bigram = words[i-1] + ' ' + words[i]
                counts[bigram] += 1
                context_counts[words[i-1]] += 1

                counts[words[i]] += 1
                context_counts[""] += 1

                counts[words[i-1] + ' ' +  words[i]] += 1 # Add bigram and bigram context
                context_counts[words[i-1]] += 1
                counts[words[i]] += 1    # Add unigram and unigram context
                context_counts[""] += 1

                pass

    # Save probabilities to the model file
    with open(model_file, 'w') as fo:
        for ngram, count in counts.items():
            # TODO: Write code to calculate probabilities of n-grams
            # (unigrams and bigrams)
            # Hint: probabilities of n-grams will be calculated by their counts
            # divided by their context's counts.
            # probability = counts[ngram]/context_counts[context]
            # After calculating probabilities, we will save ngram and probability
            # to the file in the format:
            # ngram<tab>probability

            # YOUR CODE HERE
            words = ngram.split(' ')
            words.pop()
            context = ' '.join(words)

            probability = count / context_counts[context]
            fo.write('%s\t%f\n' % (ngram, probability))

            words = ngram.split(' ')
            words.pop()
            context = ' '.join(words)
            probability = counts[ngram]/context_counts[context]
            fo.write('%s\t%f\n' % (ngram, probability))
            pass

Let's try to train bigram model on the small data.

In [34]:
train_bigram('02-train-input.txt', '02-bigram_model.txt')

Let's see the content of the model. After completing the function `train_bigram`, you should see. The order of lines may be different.

```
</s>	0.250000
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
b d	0.500000
c	0.125000
c </s>	1.000000
d	0.125000
d </s>	1.000000
```

In [23]:
# !cat 02-bigram_model.txt

In [35]:
with open("02-train-input.txt", "r", encoding="utf-8") as f:
    print(f.read())

a b c
a b d



### Part 1.2: load the model file

We are going to implement the function `load_bigram_model` to load the model file.

In [36]:
def load_bigram_model(model_file):
    """Load the model file

    Args:
        model_file (str): Path to the model file

    Returns:
        probs (dict): Dictionary object that map from ngrams to their probabilities
    """
    probs = {}
    with open(model_file, 'r') as f:
        for line in f:
            # TODO: From each line split ngram, probability
            # and then update probs

            # YOUR CODE HERE
            line = line.strip()
            if line == '':
                continue
            ngram, p = line.split('\t')
            probs[ngram] = float(p)
            pass
    return probs

Let's test the function

In [37]:
probs = load_bigram_model('02-bigram_model.txt')
probs

{'<s> a': 1.0,
 'a': 0.25,
 'a b': 1.0,
 'b': 0.25,
 'b c': 0.5,
 'c': 0.125,
 'c </s>': 1.0,
 '</s>': 0.25,
 'b d': 0.5,
 'd': 0.125,
 'd </s>': 1.0}

## Part 2: Evaluating Bigram Language Model

In this part, we will evaluate the bigram language model on the test set. We will use linear interpolation as the smoothing technique.

What we need to do is to complete the function  `test_bigram` as follows. The function will return perplexity on the test data.

Recall that, the smoothed bigram probabilities using interpolation technique is calculated as follows.

Bigrams:

$$
P(w_i|w_{i-1})=\lambda_2 \times P_{ML}(w_i|w_{i-1})+(1-\lambda_2)\times P(w_i)
$$

where $P(w_i)$ is the smoothed unigram probability and calculated as follows.

$$
P(w_i)=\lambda_1 \times P_{ML}(w_i) + (1-\lambda_1) \times \frac{1}{N}
$$

where $N$ is a large number, e.g., $N=1,000,000$.

In [38]:
import math


def test_bigram(test_file, model_file, lambda2=0.95, lambda1=0.95, N=1000000):
    W = 0 # Total word count
    H = 0
    probs = load_bigram_model(model_file)
    with open(test_file, encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')
            for i in range(1, len(words)):  # Note: starting at 1, after <s>
                # TODO: Write code to calculate smoothed unigram probabilties
                # and smoothed bigram probabilities
                # You should use calculate p1 as smoothed unigram probability
                # and p2 as smoothed bigram probability
                p1 = None  # Comment out this line
                p2 = None  # Comment out this line

                # YOUR CODE HERE

                p1 = (1-lambda1)/N   # Smoothed unigram probability
                if words[i] in probs:
                    p1 += lambda1 * probs[words[i]]
                p2 = (1-lambda2) * p1  # Smoothed bigram probability
                if words[i-1] + ' ' + words[i] in probs:
                    p2 += lambda2 * probs[words[i-1] + ' ' + words[i]]

                # END OF YOUR CODE
                W += 1  # Count the words
                H += -math.log2(p2) # We use logarithm to avoid underflow
    H = H/W
    P = 2**H

    print("Entropy: {}".format(H))
    print("Perplexity: {}".format(P))

    return P

Now let's calculate on the Wikipedia data.

In [39]:
train_bigram('wiki-en-train.word', 'bigram_model.txt')

test_bigram('wiki-en-test.word', 'bigram_model.txt')


UnicodeEncodeError: 'charmap' codec can't encode character '\u2192' in position 4: character maps to <undefined>

## Text Generation

In [ ]:
from collections import defaultdict

def build_bigram_distribution(probs):
    bigram = defaultdict(dict)

    for key, p in probs.items():
        parts = key.split()
        if len(parts) == 2:          # chỉ lấy bigram
            w_prev, w_next = parts
            bigram[w_prev][w_next] = p

    return bigram

In [ ]:
probs = load_bigram_model('bigram_model.txt')
bigram = build_bigram_distribution(probs)

In [ ]:
import random

def random_choice(distribution):
    r = random.random()
    cumulative = 0.0
    last_item = None

    for item, prob in distribution.items():
        cumulative += prob
        last_item = item
        if r <= cumulative:
            return item

    return last_item

In [ ]:
def generate_text(bigram, max_length=20):
    current = '<s>'
    result = []

    for _ in range(max_length):
        if current not in bigram:
            break

        next_word = random_choice(bigram[current])

        if next_word == '</s>':
            break

        result.append(next_word)
        current = next_word

    return ' '.join(result)


In [ ]:
for _ in range(10):
    print(generate_text(bigram))

The examples .
A machine translation , which are capable of a team led to texts by domain .
This has improved their answers the language to Australia .
Back-End or classified into machine-encoded text is prone to be the European group of these databases .
... .
In English sentences , to discover these words in the POS tagger on unsupervised and the question answering systems such
However , since 1965 it is very limited in the harmonic mean word , there are 9 parts of the
Our brain is not the top T vertices\/unigrams are the second edition published at least five years development has been
Both QA systems are spoken text , it -RRB- securely ; and computational linguistics is the final letter to be
This is usually can benefit from section requires an inseparable part of the social psychology Response based on the keyphrases
